In [1]:
import os
import time
import json
import librosa
import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import cosine
from datasets import load_dataset
from pathlib import Path

import torch
import torchaudio

In [2]:
########################################################
# Hugging face login
#######################################################
import logging, warnings
from transformers import logging as hf_logging
from huggingface_hub import login

# Silence transformers/TRL logs early
hf_logging.set_verbosity_error()
logging.getLogger("trl").setLevel(logging.ERROR)

# Hide specific noisy warnings
warnings.filterwarnings(
    "ignore",
    message=r".*loss_type=None.*ForCausalLMLoss.*",
    category=UserWarning,
)
warnings.filterwarnings(
    "ignore",
    message=r".*cuDNN SDPA backward got grad_output\.strides\(\) != output\.strides\(\).*",
    category=UserWarning,
)
os.environ["TQDM_NOTEBOOK"] = "0"

# setting key in secrets google colab
# from google.colab import userdata
# hf_key = userdata.get('HUGGINGFACE_API_KEY')
# Load .env file (if present)
from dotenv import load_dotenv
load_dotenv()
hf_key = os.environ.get("HUGGINGFACE_API_KEY")
if hf_key:
    login(hf_key)
else:
    raise EnvironmentError("HUGGINGFACE_API_KEY not found. Copy .env.template to .env and add your token. See Instruction.md")

/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
#######################################################
# CONFIGURATION
#######################################################

# Set your models here
TTS_MODELS = [
    # "bark_small", # "suno/bark-small",                                  # expressive
    "speecht5_tts", # "microsoft/speecht5_tts"                            # optional 4th model
    "mms_tts"
]

models = {}

# Directory to store generated audio
OUTPUT_DIR = Path("tts_results")
OUTPUT_DIR.mkdir(exist_ok=True)

# Example dataset containing text prompts
# DATASET_NAME = "MikhailT/lj-speech"
# Original LJ Speech dataset (popular dataset for benchmarking), 13100 short audio clips of a single female speaker
DATASET_PATH = 'mathurinache/the-lj-speech-dataset/versions/1/LJSpeech-1.1'
DATASET_NAME = 'LJSpeech1_1'
NUM_SAMPLES = 10   # subset for fast evaluation

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"


---
## suno/bark-small

In [4]:
# It is important to install numpy before torch (test this works)
# import torch
# import numpy as np

# x = torch.randn(3)
# print(x.numpy())

# Load model
from transformers import AutoProcessor, AutoModelForTextToWaveform

bark_small_local_path="./models/bark_small_local"

bark_small_local=True

# Load processor and model
try:        
    processor = AutoProcessor.from_pretrained(bark_small_local_path if bark_small_local else "suno/bark-small")
    model = AutoModelForTextToWaveform.from_pretrained(bark_small_local_path if bark_small_local else "suno/bark-small").to(device)
except OSError:
    raise EnvironmentError("Make sure that the local path to the model is correct.")
else:
    models['bark_small'] = {"model": model, "processor": processor}

/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [6]:
# save the model locally
# save_path = "./models/bark_small_local"
# model.save_pretrained(save_path)
# processor.save_pretrained(save_path)

### Testing

In [ ]:
# Test model
# Example text
text = "Hello! This is a test of Bark-small text-to-speech. I have downloaded the model locally"

# Preprocess input
inputs = processor(text=text, return_tensors="pt").to(device)

# Generate waveform
with torch.no_grad():
    waveform = model.generate(**inputs)

# waveform is a tensor with shape [num_samples]
# Save to WAV file
output_path = "bark_output.wav"
torchaudio.save(output_path, waveform.squeeze(0).cpu().unsqueeze(0), sample_rate=24000)

print(f"Saved Bark-small output to {output_path}")

Saved Bark-small output to bark_output.wav


In [12]:
from IPython.display import Audio, display
display(Audio(output_path))

---
## Microsoft/speecht5_tts

In [4]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

speecht5_tts_local_path = './models/speecht5_tts'
speecht5_tts_local = True
# Load models
try:
    processor = SpeechT5Processor.from_pretrained(speecht5_tts_local_path if speecht5_tts_local else "microsoft/speecht5_tts")
    model = SpeechT5ForTextToSpeech.from_pretrained(speecht5_tts_local_path if speecht5_tts_local else "microsoft/speecht5_tts")
    vocoder = SpeechT5HifiGan.from_pretrained(os.path.join(speecht5_tts_local_path, "vocoder") if speecht5_tts_local else "microsoft/speecht5_hifigan")
except OSError:
    raise EnvironmentError("Make sure that the local path to the model is correct.")
else:
    models["speecht5_tts"] = {"processor": processor, "model": model, "vocoder": vocoder}

/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# save the model locally
# save_path = "./models/speecht5_tts"
# models["speecht5_tts"]['model'].save_pretrained(save_path)
# models["speecht5_tts"]['processor'].save_pretrained(save_path)
# models["speecht5_tts"]['vocoder'].save_pretrained(os.path.join(save_path, 'vocoder'))

### Testing

In [12]:
# Test the model
import soundfile as sf

# Prepare input text
inputs = processor(text="Hello, this is a SpeechT5 text-to-speech test, I am super fast, god dammit", return_tensors="pt")

# Use a random speaker embedding (512 dimensions)
speaker_embeddings = torch.randn(1, 512)

# Generate speech
with torch.no_grad():
    speech = model.generate_speech(
        inputs["input_ids"],
        speaker_embeddings,
        vocoder=vocoder
    )

# Save output
sf.write("speech.wav", speech.numpy(), 16000)

In [13]:
from IPython.display import Audio, display
display(Audio('speech.wav'))

---

---
## facebook/mms-tts-eng

In [5]:
from transformers import VitsModel, AutoTokenizer

mms_tts_local_path = './models/mms_tts'
mms_tts_local = False
# Load models
try:
    model = VitsModel.from_pretrained(mms_tts_local_path if mms_tts_local else "facebook/mms-tts-eng")
    tokenizer = AutoTokenizer.from_pretrained(mms_tts_local_path if mms_tts_local else "facebook/mms-tts-eng")
except OSError:
    raise EnvironmentError("Make sure that the local path to the model is correct.")
else:
    models["mms_tts"] = {"model": model, "tokenizer": tokenizer}

/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [47]:
# # save the model locally
# save_path = "./models/mms_tts"
# models["mms_tts"]['model'].save_pretrained(save_path)
# models["mms_tts"]['tokenizer'].save_pretrained(save_path)

### Testing

In [ ]:
import torch

text = "some example text in the English language"
inputs = models['mms_tts']['tokenizer'](text, return_tensors="pt")

with torch.no_grad():
    output = models['mms_tts']['model'](**inputs).waveform

In [50]:
from IPython.display import Audio

Audio(output.numpy(), rate=models['mms_tts']['model'].config.sampling_rate)

---
## Dataset loading

In [6]:
import pandas as pd
from datasets import Dataset, load_dataset

# 1. Load metadata
df = pd.read_csv(
    f"./datasets/{DATASET_PATH}/metadata.csv",
    sep="|",
    names=["id", "text", "normalized"],
)

# 2. Add full audio paths
df["audio"] = df["id"].apply(
    lambda x: f"./datasets/{DATASET_PATH}/wavs/{x}.wav"
)

# 3. Rename fields to match your benchmark code
df = df.rename(columns={"text": "spoken_text"})

# 4. Create HuggingFace dataset
dataset = Dataset.from_pandas(df)

# 5. Tell HF that "audio" contains audio files
dataset = dataset.cast_column("audio", 
    load_dataset("audiofolder", data_dir=f"./datasets/{DATASET_PATH}/wavs")["train"].features["audio"]
)

Resolving data files:   0%|          | 0/13100 [00:00<?, ?it/s]

In [ ]:
# How originaly ljspeech was downloaded from Kagglehub
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("mathurinache/the-lj-speech-dataset")

# print("Path to dataset files:", path)

In [ ]:
# alternative dataset on huggingface for ljspeech
# DATASET_NAME = "MikhailT/lj-speech"
# # load dataset
# from datasets import load_from_disk
# import datasets

# dataset_local = True
# if dataset_local:
#     dataset = load_from_disk(os.path.join('datasets', DATASET_NAME))
# else:
#     dataset = load_dataset(DATASET_NAME, split="full", streaming=False).select(range(NUM_SAMPLES))

# dataset = dataset.cast_column("audio", datasets.features.Audio(decode=False))

# save dataset locally    
# dataset.save_to_disk(f'./datasets/{DATASET_NAME}')

In [7]:
BATCH_SIZES = [1, 5]

In [ ]:
#######################################################
# METRICS — Small, simple (expand as needed)
#######################################################
import soundfile as sf

def audio_duration(path):
    """Returns duration in seconds."""
    y, sr = librosa.load(path, sr=None)
    return len(y) / sr

# def mel_spectrogram_similarity(ref_path, gen_path):
#     """
#     Simple similarity metric comparing mel spectrogram cosine similarity.
#     Not perfect, but useful for midterm presentation.
#     """
#     ref, sr_ref = librosa.load(ref_path, sr=22050)
#     gen, sr_gen = librosa.load(gen_path, sr=22050)

#     ref_mel = librosa.feature.melspectrogram(ref, sr=22050)
#     gen_mel = librosa.feature.melspectrogram(gen, sr=22050)

#     ref_vec = np.mean(ref_mel, axis=1)
#     gen_vec = np.mean(gen_mel, axis=1)

#     return 1 - cosine(ref_vec, gen_vec)


#######################################################
# MAIN PIPELINE
#######################################################
def run_tts_benchmark(dataset):
    results_summary = {}

    for model_name in TTS_MODELS:
        print(f"\n### Running inference for: {model_name}")
        model_dir = OUTPUT_DIR / model_name.replace("/", "_")
        model_dir.mkdir(exist_ok=True)

        # infer_fn = load_tts_model(model_name)

        model_results = []
        warmup_text = "Warm up the model."
        print("Running warm-up…")
        if model_name == "speecht5_tts":
          inputs = models[model_name]["processor"](text=warmup_text, return_tensors="pt")
          speaker_embeddings = torch.zeros(1, 512)
          with torch.no_grad():
              _ = models[model_name]["model"].generate_speech(
							inputs["input_ids"],
							speaker_embeddings,
							vocoder=models[model_name]["vocoder"]
						)
        elif model_name == "mms_tts":
          inputs = models['mms_tts']['tokenizer'](warmup_text, return_tensors="pt")
          with torch.no_grad():
              _ = models['mms_tts']['model'](**inputs).waveform

        print("Warm-up complete.\n")
        

        for batch_size in BATCH_SIZES:
          print(f"Batch size: {batch_size}")

          model_results = []
          samples = list(dataset)

          if (model == 'speecht5_tts' and batch_size != 1):
             print('speecht5_tts does not support batching, skip...')
             continue

          for b_start in tqdm(range(0, len(samples), batch_size)):
              batch = samples[b_start:b_start+batch_size]
              texts = [s["spoken_text"] for s in batch]

              # ----- Inference -----
              t0 = time.time()

              if model_name == 'speecht5_tts':
                inputs = models[model_name]['processor'](text=texts[0], return_tensors="pt")
                # Use a random speaker embedding (512 dimensions)
                # fix voice for the moment
                # speaker_embeddings = torch.randn(1, 512)
                speaker_embeddings = torch.zeros(1, 512)

                # Generate speech
                with torch.no_grad():
                    generated = models[model_name]['model'].generate_speech(
                        inputs["input_ids"],
                        speaker_embeddings,
                        vocoder=models[model_name]['vocoder']
                    )
                
                generated_batch = generated
                
              # elif model_name == 'bark_small':
              #   # Preprocess input
              #   inputs = models[model_name]['processor'](text=text, return_tensors="pt").to(device)

              #   # Generate waveform
              #   with torch.no_grad():
              #       generated_audio = models[model_name]['model'].generate(**inputs)
              elif model_name == 'mms_tts':
                inputs = models['mms_tts']['tokenizer'](text=texts, return_tensors="pt", padding=True)

                with torch.no_grad():
                  output = models['mms_tts']['model'](**inputs).waveform
                
                generated_batch = output

              else:
                Warning("Model not implemented yet!")
                return

              t1 = time.time()

              if model_name == 'speecht5_tts':
                output_audio_path = model_dir / f"sample_{b_start}_bs{batch_size}.wav"
                sf.write(output_audio_path, generated_batch.numpy(), 16000)
                # Metrics
                duration = audio_duration(output_audio_path)
                # similarity = mel_spectrogram_similarity(reference_audio_path,output_audio_path)

                model_results.append({
                    "batch_size": batch_size,
                    "text": batch[0]["spoken_text"],
                    "reference": batch[0]["audio"]["path"],
                    "generated": str(output_audio_path),
                    "inference_time": t1 - t0,   # batch time (not per-sample)
                    "duration": duration
                    # "mel_similarity": float(similarity)
                })
                continue

              # Save generated audio
              for idx, (sample, audio) in enumerate(zip(batch, generated_batch)):
                output_audio_path = model_dir / f"sample_{b_start + idx}_bs{batch_size}.wav"
                # elif model_name == 'bark_small':
                #   # waveform is a tensor with shape [num_samples]
                #   torchaudio.save(output_audio_path, audio.squeeze(0).cpu().unsqueeze(0), sample_rate=24000)
                if model_name == 'mms_tts':
                  waveform = audio.cpu().numpy()
                  sf.write(output_audio_path, waveform, models['mms_tts']['model'].config.sampling_rate)

                # Metrics
                duration = audio_duration(output_audio_path)
                # similarity = mel_spectrogram_similarity(reference_audio_path,output_audio_path)

                model_results.append({
                    "batch_size": batch_size,
                    "text": sample["spoken_text"],
                    "reference": sample["audio"]["path"],
                    "generated": str(output_audio_path),
                    "inference_time": t1 - t0,   # batch time (not per-sample)
                    "duration": duration
                    # "mel_similarity": float(similarity)
                })

          # Save model results
          with open(model_dir / f"results_bs{batch_size}.json", "w") as f:
              json.dump(model_results, f, indent=2)

          # # Summary per model
          # avg_time = np.mean([r["inference_time"] for r in model_results])
          # # avg_sim = np.mean([r["mel_similarity"] for r in model_results])
          # avg_len = np.mean([r["duration"] for r in model_results])

          # results_summary[model_name] = {
          #     "avg_inference_time": float(avg_time),
          #     # "avg_mel_similarity": float(avg_sim),
          #     "avg_duration_s": float(avg_len)
          # }

    # Save cross-model summary
    # with open(OUTPUT_DIR / "summary.json", "w") as f:
    #     json.dump(results_summary, f, indent=2)

    print("\nDone! Summary saved in tts_results/summary.json")

In [9]:
dataset_sampled = dataset.select(range(50))
run_tts_benchmark(dataset_sampled)
print(TTS_MODELS)


### Running inference for: speecht5_tts
Running warm-up…
Warm-up complete.

Batch size: 1


/home/kaeslin/miniconda3/envs/msgai/lib/python3.10/site-packages/librosa/core/intervals.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
100%|██████████| 50/50 [02:59<00:00,  3.60s/it]


Batch size: 5


100%|██████████| 10/10 [00:33<00:00,  3.33s/it]



### Running inference for: mms_tts
Running warm-up…
Warm-up complete.

Batch size: 1


100%|██████████| 50/50 [02:03<00:00,  2.47s/it]


Batch size: 5


100%|██████████| 10/10 [02:39<00:00, 15.95s/it]


Done! Summary saved in tts_results/summary.json
['speecht5_tts', 'mms_tts']


In [ ]:
import pandas as pd
# df_summary = pd.read_json('tts_results/summary.json')
os.listdir('tts_results/mms_tts')
df_mms_tts_b1 = pd.read_json('tts_results/mms_tts/results_bs1.json')
df_mms_tts_b5 = pd.read_json('tts_results/mms_tts/results_bs5.json')
df_speecht5_tts_b1 = pd.read_json('tts_results/speecht5_tts/results_bs1.json')

In [38]:
df_mms_tts_b1.head(5)
df_mms_tts_b1['rtf'] = df_mms_tts_b1['inference_time']/df_mms_tts_b1['duration']
df_mms_tts_b1.describe()

,batch_size,inference_time,duration,rtf
count,50.0,50.000000,50.00000,50.000000
mean,1.0,2.456845,6.40736,0.383212
std,0.0,0.889313,2.13284,0.044410
min,1.0,0.436975,1.18400,0.272729
25%,1.0,1.833652,4.87200,0.356106
50%,1.0,2.486406,6.48800,0.378647
75%,1.0,3.000873,7.84400,0.415977
max,1.0,4.527133,10.84800,0.510611


In [39]:
df_mms_tts_b5.head(5)
df_mms_tts_b5['rtf'] = df_mms_tts_b5['inference_time']/df_mms_tts_b5['duration']
df_mms_tts_b5.describe()

,batch_size,inference_time,duration,rtf
count,50.0,50.000000,50.0000,50.000000
mean,5.0,15.909744,8.9360,1.786932
std,0.0,1.759357,1.0415,0.124774
min,5.0,13.348673,6.8640,1.511660
25%,5.0,14.385648,8.0960,1.760720
50%,5.0,15.808713,9.2400,1.788776
75%,5.0,17.240968,9.9040,1.879285
max,5.0,19.229020,10.0800,1.944737


In [40]:
df_speecht5_tts_b1.head(5)
df_speecht5_tts_b1['rtf'] = df_speecht5_tts_b1['inference_time']/df_speecht5_tts_b1['duration']
df_speecht5_tts_b1.describe()

,batch_size,inference_time,duration,rtf
count,50.0,50.000000,50.000000,50.000000
mean,1.0,3.585220,3.911680,0.931017
std,0.0,1.280228,1.381745,0.138318
min,1.0,0.369606,0.256000,0.718504
25%,1.0,2.742196,3.120000,0.845843
50%,1.0,3.653893,4.096000,0.902577
75%,1.0,4.415739,4.592000,1.016577
max,1.0,5.940861,7.104000,1.443773
